In [1]:
import os
import glob

In [2]:
ip_dict = {"individual_1": ['brain_IP_1', 'heart_IP_1', 'liver_IP_1'],  
            "individual_2": ['brain_IP_2', 'heart_IP_2', 'pla_IP_2', 'kid_IP_2', 'liver_IP_2'],  
            "individual_3": ['brain_IP_3', 'heart_IP_3', 'liver_IP_3', 'kid_IP_3'], 
            "individual_4": ['sto_IP_4', 'mus_IP_4', 'lung_IP_4', 'kid_IP_4', 'pla_IP_4'], 
            "individual_5": ['sto_IP_5', 'mus_IP_5', 'lung_IP_5'], 
            "individual_6": ['pla_IP_6'],
            "individual_7": ['CRR073020', 'CRR042306', 'CRR042296', 'CRR042318', 'CRR042310', 'CRR042300', 'CRR042280', 'CRR042312', 'CRR055561', 'CRR055551', 'CRR055563', 'CRR042302', 'CRR042294', 'CRR042290'], 
            "individual_8": ['CRR042308', 'CRR042286', 'CRR042282', 'CRR042284', 'CRR042298', 'CRR042316', 'CRR042314', 'CRR055557', 'CRR055555', 'CRR042288', 'CRR055559', 'CRR042320', 'CRR042292'], 
            "individual_9": ['CRR073018', 'CRR055537', 'CRR055536', 'CRR055539', 'CRR055533'],
            "individual_10": ['CRR055549', 'CRR055547', 'CRR055545', 'CRR055543', 'CRR055541'],
            "individual_11": ['CRR055525', 'CRR055527', 'CRR042278', 'CRR055529', 'CRR055531', 'CRR042304'], 
            "individual_12": ['CRR055553'],
            "individual_13": ['CRR073016'],
           }
           
           
input_dict = {"individual_1": ['brain_1_L6', 'heart_1_L6', 'liver_1_L6'],  
            "individual_2": ['brain_2_L7', 'heart_2_L7', 'pla_2_L3', 'kid_2_L7', 'liver_2_L7'],  
            "individual_3": ['brain_3_L68', 'heart_3_L6', 'liver_3_L6', 'kid_3_L8'], 
            "individual_4": ['sto_4_L3', 'mus_4_L6', 'lung_4_L7', 'kid_4_L6', 'pla_4_L6'], 
            "individual_5": ['sto_5_L7', 'mus_5_L7', 'lung_5_L7'], 
            "individual_6": ['pla_6_L6'],
            "individual_7": ['CRR073021', 'CRR042307', 'CRR042297', 'CRR042319', 'CRR042311', 'CRR042301', 'CRR042281', 'CRR042313', 'CRR055562', 'CRR055552', 'CRR055564', 'CRR042303', 'CRR042295', 'CRR042291'], 
            "individual_8": ['CRR042309', 'CRR042287', 'CRR042283', 'CRR042285', 'CRR042299', 'CRR042317', 'CRR042315', 'CRR055558', 'CRR055556', 'CRR042289', 'CRR055560', 'CRR042321', 'CRR042293'], 
            "individual_9": ['CRR073019', 'CRR055538', 'CRR055535', 'CRR055540', 'CRR055534'],
            "individual_10": ['CRR055550', 'CRR055548', 'CRR055546', 'CRR055544', 'CRR055542'],
            "individual_11": ['CRR055526', 'CRR055528', 'CRR042279', 'CRR055530', 'CRR055532', 'CRR042305'], 
            "individual_12": ['CRR055554'],
            "individual_13": ['CRR073017'],
           }

In [13]:
expre_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/binomial_test/sig/"
exp_list = glob.glob("%s/*.bed" % expre_dir)  # brain_1_L6.binomials.txt

m6a_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/"
m6a_list = glob.glob("%s/*.bed" % m6a_dir)   # brain_IP_1.txt

result_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/"
os.system("mkdir -p %s" % result_dir)

def process_m6a(m6a):
    df = pd.read_table(m6a, sep="\t")
    df['m6a_difference'] = df['altRPKM_ratio'] - df['refRPKM_ratio']
    df['m6a_mark'] = np.where(df['m6a_difference']>0, "alt", "ref")
    selected_cols = ['contig','position','refAllele','altAllele','m6a_mark','m6a_difference']
    df_sub = df[selected_cols]
    return df_sub

def process_expression(expression):
    df = pd.read_table(expression, sep="\t")
    df['expression_difference'] = df['altCount'] - df['refCount']
    df['expression_mark'] = np.where(df['expression_difference']>0, "alt", "ref")
    selected_cols = ['contig','position','refAllele','altAllele','expression_mark','expression_difference']
    df_sub = df[selected_cols]
    return df_sub

oppo_dir = result_dir + os.sep + "oppo"
same_dir = result_dir + os.sep + "same"
os.system("mkdir -p %s" % oppo_dir)
os.system("mkdir -p %s" % same_dir)

for ind, prefix_list in ip_dict.items():
    for i in range(len(prefix_list)):
        m6a = os.path.join(m6a_dir, "%s.txt" % prefix_list[i])
        input_prefix = input_dict[ind][i]
        expression = os.path.join(expre_dir, "%s.binomials.txt" % input_prefix)
        df_m6a, df_expression = process_m6a(m6a), process_expression(expression)
        keys = ['contig','position','refAllele','altAllele']
        df=df_m6a.merge(df_expression,on=keys,how='left').drop_duplicates().dropna()
        print(prefix_list[i], input_prefix)
        df['term'] = df['contig'] + ":" + df['position'].astype(str)
        selected_cols = ['term','m6a_mark','m6a_difference','expression_mark','expression_difference']
        df_sub = df[selected_cols]
        # print(df_sub.head())
        ## 
        df_oppo = df_sub[df_sub['m6a_mark'] != df_sub['expression_mark']]
        result_file = os.path.join(oppo_dir, "%s.txt" % input_prefix)
        df_oppo.to_csv(result_file, sep="\t", index=False)
        #
        df_same = df_sub[df_sub['m6a_mark'] == df_sub['expression_mark']]
        result_file = os.path.join(same_dir, "%s.txt" % input_prefix)
        df_same.to_csv(result_file, sep="\t", index=False)
        # df_sub.to_csv(result_file, sep="\t", index=False)

/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/brain_IP_1.txt brain_1_L6
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/heart_IP_1.txt heart_1_L6
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/liver_IP_1.txt liver_1_L6
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/brain_IP_2.txt brain_2_L7
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/heart_IP_2.txt heart_2_L7
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/pla_IP_2.txt pla_2_L3
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/kid_IP_2.txt kid_2_L7
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/liver_IP_2.txt liver_2_L7
/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysi